<a href="https://colab.research.google.com/github/Aravind-Sridhar/NLP_Implementation/blob/main/4_Fake_News_Classifier_Using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [31]:
import pandas as pd

In [32]:
df=pd.read_csv('train.csv')

In [33]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [34]:
df.shape

(20800, 5)

In [35]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [36]:
###Drop Nan Values
df=df.dropna()

In [ ]:
df.shape

(18285, 5)

In [37]:
## Get the Independent Features
X=df.drop('label',axis=1)

In [38]:
## Get the Dependent features
y=df['label']

In [39]:
X.shape

(18285, 4)

In [40]:
y.shape

(18285,)

In [41]:
import tensorflow as tf

In [42]:
tf.__version__

'2.11.0'

In [43]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [44]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [51]:
messages=X.copy()

In [52]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [53]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [59]:
messages.reset_index(inplace=True)

In [60]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [61]:
import nltk
import re
from nltk.corpus import stopwords

In [62]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [63]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [64]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [65]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [66]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[1257, 2966, 1975, 4173, 2561, 488, 242, 1482, 723, 4753],
 [4987, 514, 2399, 2806, 4622, 2656, 1477],
 [4149, 3673, 2873, 1913],
 [3929, 4285, 4966, 3491, 2784, 1408],
 [3105, 4622, 1672, 1856, 4816, 1625, 4622, 3827, 3178, 324],
 [1100,
  3652,
  1095,
  3273,
  2861,
  485,
  1007,
  4395,
  2031,
  4383,
  2955,
  533,
  4648,
  1551,
  1477],
 [1364, 163, 4139, 1635, 1408, 1754, 3557, 502, 1781, 3985, 1353],
 [4363, 2913, 4077, 109, 1185, 321, 485, 4493, 1781, 3985, 1353],
 [2336, 2671, 1949, 524, 2610, 4259, 1123, 3776, 485, 2540],
 [4070, 3999, 125, 3154, 1399, 1487, 1943, 545],
 [98, 4241, 985, 3357, 3135, 3781, 4800, 2777, 103, 4159, 3365],
 [3491, 1576, 2561, 4259, 485, 1185],
 [3852, 460, 4167, 1506, 1073, 114, 3606, 4838, 1721],
 [1438, 1078, 3038, 334, 356, 4913, 2985, 1781, 3985, 1353],
 [911, 628, 1022, 1714, 4652, 1781, 3985, 1353],
 [4668, 757, 1140, 284, 886, 2518, 1655, 1730, 1665, 4014],
 [4559, 2993, 514],
 [1468, 1140, 403, 1062, 485, 1211, 4641, 1477],
 [652, 29

In [67]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [68]:
onehot_repr[1]

[4987, 514, 2399, 2806, 4622, 2656, 1477]

### Embedding Representation

In [69]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[1257 2966 1975 ...    0    0    0]
 [4987  514 2399 ...    0    0    0]
 [4149 3673 2873 ...    0    0    0]
 ...
 [ 189 4742 4437 ...    0    0    0]
 [4847 1185 2902 ...    0    0    0]
 [4053 2563  842 ...    0    0    0]]


In [70]:
embedded_docs[1]

array([4987,  514, 2399, 2806, 4622, 2656, 1477,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [71]:
embedded_docs[0]

array([1257, 2966, 1975, 4173, 2561,  488,  242, 1482,  723, 4753,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [72]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [73]:
len(embedded_docs),y.shape

(18285, (18285,))

In [74]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [75]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [76]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [99]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=20,batch_size=64)

Epoch 1/20
192/192 [==============================] - 12s 45ms/step - loss: 0.3232 - accuracy: 0.8432 - val_loss: 0.2010 - val_accuracy: 0.9158
Epoch 2/20
192/192 [==============================] - 3s 16ms/step - loss: 0.1518 - accuracy: 0.9434 - val_loss: 0.1936 - val_accuracy: 0.9239
Epoch 3/20
192/192 [==============================] - 3s 13ms/step - loss: 0.1152 - accuracy: 0.9562 - val_loss: 0.2151 - val_accuracy: 0.9150
Epoch 4/20
192/192 [==============================] - 2s 10ms/step - loss: 0.0877 - accuracy: 0.9698 - val_loss: 0.2725 - val_accuracy: 0.9085
Epoch 5/20
192/192 [==============================] - 2s 11ms/step - loss: 0.0728 - accuracy: 0.9744 - val_loss: 0.2496 - val_accuracy: 0.9152
Epoch 6/20
192/192 [==============================] - 2s 9ms/step - loss: 0.0565 - accuracy: 0.9819 - val_loss: 0.2433 - val_accuracy: 0.9064
Epoch 7/20
192/192 [==============================] - 2s 8ms/step - loss: 0.0453 - accuracy: 0.9847 - val_loss: 0.3663 - val_accuracy: 0.9142


### Adding Dropout 

In [92]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [100]:
y_pred=model.predict(X_test)

189/189 [==============================] - 0s 2ms/step


In [101]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [102]:
from sklearn.metrics import confusion_matrix

In [103]:
confusion_matrix(y_test,y_pred)

array([[3150,  269],
       [ 299, 2317]])

In [104]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9058823529411765

In [105]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.91      0.92      0.92      3419
           1       0.90      0.89      0.89      2616

    accuracy                           0.91      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.91      0.91      0.91      6035

